<a href="https://colab.research.google.com/github/r-kovalch/acter-ner/blob/main/notebooks/acter-rerank-min-roberta-large-en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!git clone https://github.com/Reennon/acter-ner

Cloning into 'acter-ner'...
remote: Enumerating objects: 522, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 522 (delta 11), reused 3 (delta 3), pack-reused 491 (from 1)
Receiving objects: 100% (522/522), 4.19 MiB | 8.92 MiB/s, done.
Resolving deltas: 100% (337/337), done.


In [2]:
!git clone https://github.com/AylaRT/ACTER

Cloning into 'ACTER'...
remote: Enumerating objects: 5448, done.
remote: Counting objects: 100% (5448/5448), done.
remote: Compressing objects: 100% (3304/3304), done.
remote: Total 5448 (delta 2684), reused 4893 (delta 2132), pack-reused 0 (from 0)
Receiving objects: 100% (5448/5448), 11.90 MiB | 14.64 MiB/s, done.
Resolving deltas: 100% (2684/2684), done.


In [3]:
%cd /content/ACTER

/content/ACTER


In [4]:
%cd /content/acter-ner/term_extractor

/content/acter-ner/term_extractor


In [37]:
!rm -r train_full.tsv test_full.tsv input_data output

rm: cannot remove 'train_full.tsv': No such file or directory
rm: cannot remove 'test_full.tsv': No such file or directory
rm: cannot remove 'input_data': No such file or directory


In [38]:
!cp /content/drive/MyDrive/ucu/ner/threshold_datasets/ReRank_min_0.1_train_full.tsv train_full.tsv && \
  cp /content/drive/MyDrive/ucu/ner/threshold_datasets/val_full.tsv test_full.tsv

In [40]:
# !mkdir output && \
#   spacy convert --converter iob train_full.tsv output && \
#   spacy convert --converter iob test_full.tsv output


ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
Traceback (most recent call last):
  File "/usr/local/bin/spacy", line 10, in <module>
    sys.exit(setup_cli())
             ^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/spacy/cli/_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1442, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/typer/core.py", line 740, in main
    return _main(
           ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/typer/core.py", line 195, in _main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1830, in invoke
    return _process_result(sub_ctx.command.invoke(sub_

In [42]:
# Cell X – programmatic IOB‐TSV → DocBin conversion for multiple splits
import spacy
from spacy.tokens import DocBin, Span
from pathlib import Path

# 1) Initialize blank pipeline (no models loaded)
nlp = spacy.blank("en")

# 2) List of (input TSV, desired output .spacy) pairs
splits = {
    "train_full.tsv":   "train_full.spacy",
    "test_full.tsv": "test_full.spacy",
}

# 3) Prepare output directory
out_dir = Path("output")
out_dir.mkdir(exist_ok=True)

# 4) Conversion loop
for tsv_name, spacy_name in splits.items():
    tsv_path = Path(tsv_name)
    if not tsv_path.exists():
        print(f"⚠️  Skipping missing {tsv_name}")
        continue

    docbin = DocBin(store_user_data=True)
    doc_count = ent_count = 0

    # read token-per-line, blank lines separate sentences → we group N sentences into one Doc
    # Here we’ll group every sentence as its own Doc (n_sents=1)
    words, labels = [], []
    with tsv_path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if words:
                    doc = spacy.tokens.Doc(nlp.vocab, words=words)
                    ents = []
                    # scan IOB labels to recover Span(start, end)
                    start = None
                    for i, tag in enumerate(labels):
                        if tag.startswith("B"):
                            if start is not None:
                                ents.append(Span(doc, start, i, label="TERM"))
                            start = i
                        elif tag.startswith("I"):
                            # continuation
                            continue
                        else:  # "O" or other
                            if start is not None:
                                ents.append(Span(doc, start, i, label="TERM"))
                                start = None
                    # catch final
                    if start is not None:
                        ents.append(Span(doc, start, len(labels), label="TERM"))

                    doc.ents = ents
                    docbin.add(doc)
                    doc_count += 1
                    ent_count += len(ents)
                    words, labels = [], []
                continue

            # parse token and IOB label (label may be "O" or "B-TERM"/"I-TERM")
            parts = line.split("\t")
            if len(parts) != 2:
                continue
            tok, tag = parts
            words.append(tok)
            # normalize to plain "B"/"I"/"O"
            if tag.startswith("B"):
                labels.append("B")
            elif tag.startswith("I"):
                labels.append("I")
            else:
                labels.append("O")

        # flush last sentence if missing trailing blank line
        if words:
            doc = spacy.tokens.Doc(nlp.vocab, words=words)
            ents = []
            start = None
            for i, tag in enumerate(labels):
                if tag == "B":
                    if start is not None:
                        ents.append(Span(doc, start, i, label="TERM"))
                    start = i
                elif tag == "O" and start is not None:
                    ents.append(Span(doc, start, i, label="TERM"))
                    start = None
            if start is not None:
                ents.append(Span(doc, start, len(labels), label="TERM"))
            doc.ents = ents
            docbin.add(doc)
            doc_count += 1
            ent_count += len(ents)

    # write out the DocBin
    out_path = out_dir / spacy_name
    docbin.to_disk(out_path)
    print(f"✅ Converted {tsv_name} → {out_path} "
          f"({doc_count} docs, {ent_count} entities)")


✅ Converted train_full.tsv → output/train_full.spacy (9831 docs, 32792 entities)
✅ Converted test_full.tsv → output/test_full.spacy (11547 docs, 26651 entities)


In [44]:
!mkdir input_data && \
  mv output/train_full.spacy input_data/train_full.spacy && \
  mv output/test_full.spacy input_data/test_full.spacy

In [33]:
!pip install 'spacy[transformers]' -q

In [45]:
!spacy train configs/config_base.cfg  --gpu-id 0

ℹ No output directory provided
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
2025-05-23 00:26:05.276663: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 00:26:05.294484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747959965.315784   10009 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747959965.322337   10009 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-23 00:26:0

In [46]:
from google.colab import runtime
runtime.unassign()